In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from transformers import T5Tokenizer, T5ForConditionalGeneration
import helper_funcs
import torch

/home/mputo/.conda/envs/search/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
query = "Input your query here"

In [3]:
model_name = "doc2query/msmarco-t5-small-v1"
tokenizer = T5Tokenizer.from_pretrained(model_name)
question_generator = T5ForConditionalGeneration.from_pretrained(model_name)

In [4]:
encoder = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [5]:
d = 384
description = "Flat"

index = helper_funcs.create_index(d, description, gpu=True)

print(index.ntotal)

0


In [6]:
xb = torch.load('../msmarco-vec/msmarco-vectors-1.pt')

print(xb.shape)

In [ ]:
helper_funcs.populate_index(xb, index, to_cpu=True)

print(index.ntotal)

1000000


In [ ]:
collection = helper_funcs.tsv_to_df("../collectionandqueries/collection", col_name=["id", "paragraph"], index_col="id")

In [ ]:
r = helper_funcs.create_redis()

helper_funcs.save_collection(r, collection, index.ntotal)

In [ ]:
reranked_results = helper_funcs.search_and_rerank_redis(query, 10, encoder=encoder, cross_encoder=cross_encoder, index=index, collection=collection)

reranked_results

[((1,
   'The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.'),
  4.1254525),
 ((0,
   'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'),
  3.8581057),
 ((2,
   'Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.'),
  3.2710392),
 ((7,
   'Manhattan Project. The Manhattan Project was a research and development undertaking during World War II that produced the first nuclear weapons. It was led 

In [ ]:
answer = helper_funcs.generate_answer(query, reranked_results)

answer

{'score': 0.18403230607509613,
 'start': 453,
 'end': 504,
 'answer': 'hundreds of thousands of innocent lives obliterated'}

In [ ]:
sample_queries = helper_funcs.generate_queries(query, question_generator, tokenizer)

sample_queries